<a href="https://github.com/ternaustralia/cmrset-examples/blob/main/gee-examples/python/Example_3_Unpacking_the_CMRSET_QA_Band.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View on GitHub"/></a>
&nbsp;
<a href="https://colab.research.google.com/github/ternaustralia/cmrset-examples/blob/main/gee-examples/python/Example_3_Unpacking_the_CMRSET_QA_Band.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
&nbsp;
<a href="https://mybinder.org/v2/gh/ternaustralia/cmrset-examples/HEAD?labpath=gee-examples%2Fpython%2FExample_3_Unpacking_the_CMRSET_QA_Band.ipynb" target="_parent"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a>


# Example 3 - Unpacking the CMRSET QA Band
This example illustrates how to unpack the CMRSET QA band.  The CMRSET QA band contains 4 pieces of extra information for each pixel which may further assist with your analysis. These 4 pieces of information are: the source of the data; condensation adjustment; the number of Landsat observations used for this pixel; and whether the pixel value was derived from Landsat SLC-Off data.

To run this example it is assumed you have [registered](https://signup.earthengine.google.com/) to [Google Earth Engine](https://earthengine.google.com/). Please feel free to adapt this example to meet your own requirements.

---
**Execution Time:** Real-time

**Storage Required:** Nil

## Install geemap
The [geemap](https://geemap.org/) Python package is built upon [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages, and enables users to analyze and visualize Earth Engine datasets interactively within a Jupyter-based environment.  The geemap API also mimics the official Google Earth Engine Javascript Map API when making calls such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`, etc. This simplifies the translation between both languages if you are transitioning from Javascript to Python.

Note: the package ipyleaflet is not compatible with Google Colab, hence you will notice we specifically import the [folium implementation](https://geemap.org/foliumap/) in this example for broarder compatibility.

In [1]:
# Installs the geemap package
import subprocess

try:
    import geemap.foliumap as geemap
except ImportError:
    print('Installing geemap...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [2]:
import geemap.foliumap as geemap

## Authenticate and initialize
Run the `ee.Authenticate()` function to authenticate your access to Earth Engine servers and `ee.Initialize()` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account.  **Please follow the instructions in the prompts.** You will also need to enter your Google Earth Engine Cloud Project ID below.

In [3]:
# Import the Earth Engine module
import ee

# Cloud Project ID to associate with your Earth Engine workspace.
project = "Your Project ID to use"

try:
    ee.Initialize(project=project) # Initialize the library.
except Exception as e:
    ee.Authenticate() # If initialization fails, authentication is required... Trigger the authentication flow and initialize again.
    ee.Initialize(project=project) # Initialize the library.

## Unpack the pixel QA band
Let's access [CMRSET](https://developers.google.com/earth-engine/datasets/catalog/TERN_AET_CMRSET_LANDSAT_V2_2) from the [Earth Engine data catalog](https://developers.google.com/earth-engine/datasets), and filter down the [ImageCollection](https://developers.google.com/earth-engine/tutorials/tutorial_api_04) to a single month.

In [4]:
# Define the period of interest.
date = '2004-07-01'

# Access the CMRSET dataset.
cmrset_aet = ee.ImageCollection("TERN/AET/CMRSET_LANDSAT_V2_2").filterDate(date).first()

When two bytes meet in a bar... The first byte asks, “Are you ill?”
The second byte replies, “No, just feeling a bit off”.

You don't want your science data to be a bit off, so let's accurately unpack the [CMRSET pixel QA byte](https://developers.google.com/earth-engine/datasets/catalog/TERN_AET_CMRSET_LANDSAT_V2_2#bands) using this function...

In [5]:
# Helper function to unpack the pixel_qa band.
def unpack_pixel_qa(image):

  qa = image.select("pixel_qa")

  # Unpack the bits.
  data_source = qa.bitwiseAnd(3).rename('data_source') # Bits: 0-1
  condensation = qa.rightShift(2).bitwiseAnd(1).rename('condensation') # Bit: 2
  landsat_count = qa.rightShift(3).bitwiseAnd(15).rename('landsat_count') # Bits: 3-6
  slc_off = qa.rightShift(7).rename('slc_off') # Bit: 7

  # Append to the original image.
  return image.addBands(data_source).addBands(condensation).addBands(landsat_count).addBands(slc_off)

In [6]:
# Call the helper function to unpack the band.
result = unpack_pixel_qa(cmrset_aet)

## Preview the result
Let's now take a look at the four layers on a map....

In [7]:
# Visualise the layers on the map.
Map = geemap.Map(add_google_map = False)
Map.addLayer(result.select("ETa"), {'min':0, 'max':7, 'palette': ["d7191c","fdae61","ffffbf","abd9e9","2c7bb6"]}, "ETa", False)

Map.addLayer(result.select('data_source'), {'min':0, 'max': 3, 'palette': 'black,red,orange,green'}, 'Data Source (pixel_qa)');
Map.addLayer(result.select('condensation'), {'min':0, 'max': 1, 'palette': 'black,magenta'}, 'Condensation (pixel_qa)');
Map.addLayer(result.select('landsat_count'), {'min':0, 'max': 7, 'palette': 'white,blue'}, 'Landsat Count (pixel_qa)');
Map.addLayer(result.select('slc_off'), {'min':0, 'max': 1, 'palette': 'black,magenta'}, 'SLC-Off (pixel_qa)');

Map.centerObject(result)

The provided color (black,red,orange,green) is invalid. Using the default black color.
'#black,red,orange,green' is not in web format. Need 3 or 6 hex digit.
The provided color (black,magenta) is invalid. Using the default black color.
'#black,magenta' is not in web format. Need 3 or 6 hex digit.
The provided color (white,blue) is invalid. Using the default black color.
'#white,blue' is not in web format. Need 3 or 6 hex digit.
The provided color (black,magenta) is invalid. Using the default black color.
'#black,magenta' is not in web format. Need 3 or 6 hex digit.


In [8]:
Map